In [1]:
%load_ext autoreload
%autoreload 2

import os.path as osp
import sys
from mmengine.config import Config, DictAction
from mmengine.runner import Runner

from mmdet3d.datasets import *
from torch.utils.data import DataLoader
from mmdet3d.utils import register_all_modules

from mmdet3d.structures import points_cam2img
from mmdet3d.structures.bbox_3d import rotation_3d_in_axis
import cv2
import numpy as np
import torch
from torch import Tensor

In [2]:
config_file = '/home/javier/sensus-loci/sensus/configs/smoke/smoke_dla34_dlaneck_gn-all_4xb8-6x_dair-mono3d.py'

In [3]:
cfg = Config.fromfile(config_file)
register_all_modules()
train_data_cfg = cfg['train_dataloader']['dataset']
# Obtain dataset type and then delete the "type" field from the config
dataset_type = train_data_cfg.pop("type")

DatasetClass = globals()[dataset_type]
train_dataset = DatasetClass(**train_data_cfg)

/home/javier/sensus-loci/mmdetection3d/mmdet3d/evaluation/functional/kitti_utils/eval.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_thresholds(scores: np.ndarray, num_gt, num_sample_pts=41):


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
10/31 16:37:03 - mmengine - INFO - ------------------------------
10/31 16:37:03 - mmengine - INFO - The length of the dataset: 5653
10/31 16:37:03 - mmengine - INFO - The number of instances per category in the dataset:
+------------+--------+
| category   | number |
+------------+--------+
| Pedestrian | 20266  |
| Cyclist    | 13853  |
| Car        | 84073  |
+------------+--------+


In [5]:
sample = train_dataset[1]
info_3d = sample['data_samples'].gt_instances_3d
info_2d = sample['data_samples'].gt_instances
info_pts = sample['data_samples'].gt_pts_seg
info_ann = sample['data_samples'].eval_ann_info
print(sample)
cam2img = sample['data_samples'].cam2img
print(cam2img)
lidar2cam = sample['data_samples'].lidar2cam
img_path = sample['data_samples'].img_path


{'data_samples': <Det3DDataSample(

    META INFORMATION
    pad_shape: (1080, 1920, 3)
    affine_aug: True
    flip: True
    transformation_3d_flow: ['HF']
    cam2img: [[2757.839717, 0.0, 927.562803, 0.0], [0.0, 2899.073935, 576.566446, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]]
    ori_shape: (1080, 1920)
    pcd_horizontal_flip: True
    img_shape: (1080, 1920, 3)
    pcd_vertical_flip: False
    img_path: '/home/javier/datasets/DAIR/single-infrastructure-side-mmdet/training/image_2/007091.png'
    lidar2img: array([[ 9.95881576e+02, -2.74178424e+03, -1.47270853e+02,
                -6.35357353e+03],
               [ 2.89930894e+01,  2.71561219e+00, -2.55126353e+03,
                 1.82485702e+04],
               [ 9.82800841e-01,  1.27858296e-02, -1.57833755e-01,
                 9.74204898e-01],
               [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                 1.00000000e+00]])
    trans_mat: array([[ 0.17857143, -0.        ,  0.        ],
            

In [5]:
def get_pitch(transformation_matrix):
    """
    Extracts the Euler angles from a 4x4 transformation matrix.
    
    Args:
    - transformation_matrix (numpy.array): 4x4 transformation matrix

    Returns:
    - roll, pitch, yaw (floats): Euler angles in radians
    """
    
    assert transformation_matrix.shape == (4, 4), "Matrix must be 4x4"
    
    # Extract 3x3 rotation matrix from the 4x4 transformation matrix
    R = transformation_matrix[:3, :3]
    
    # Pitch
    pitch = np.arctan2(-R[2, 0], np.sqrt(R[0, 0]**2 + R[1, 0]**2)) + np.pi/2
    print(pitch)
    
    return pitch

def get_corners(bbox, lidar2cam):
    """Convert boxes to corners in clockwise order, in the form of (x0y0z0,
    x0y0z1, x0y1z1, x0y1z0, x1y0z0, x1y0z1, x1y1z1, x1y1z0).

    .. code-block:: none

                        front z
                            /
                            /
            (x0, y0, z1) + -----------  + (x1, y0, z1)
                        /|            / |
                        / |           /  |
        (x0, y0, z0) + ----------- +   + (x1, y1, z1)
                        |  /      .   |  /
                        | / origin    | /
        (x0, y1, z0) + ----------- + -------> right x
                        |             (x1, y1, z0)
                        |
                        v
                down y

    Returns:
        Tensor: A tensor with 8 corners of each box in shape (N, 8, 3).
    """
    if bbox.tensor.numel() == 0:
        return torch.empty([0, 8, 3], device=bbox.tensor.device)

    dims = bbox.dims
    corners_norm = torch.from_numpy(
        np.stack(np.unravel_index(np.arange(8), [2] * 3), axis=1)).to(
            device=dims.device, dtype=dims.dtype)

    corners_norm = corners_norm[[0, 1, 3, 2, 4, 5, 7, 6]]
    # use relative origin (0.5, 1, 0.5)
    corners_norm = corners_norm - dims.new_tensor([0.5, 1, 0.5])
    corners = dims.view([-1, 1, 3]) * corners_norm.reshape([1, 8, 3])

    pitch = get_pitch(lidar2cam)

    corners = rotation_3d_in_axis(
        corners, bbox.tensor[:, 6], axis=1)
    corners = rotation_3d_in_axis(
        corners, pitch, axis=0)
    corners += bbox.tensor[:, :3].view(-1, 1, 3)

    # Compute the geometrical center
    center = torch.mean(corners, dim=1, keepdim=True)
    
    # Concatenate the center to the corners, resulting in a shape (N, 9, 3)
    corners_with_center = torch.cat([center, corners], dim=1)
    
    return corners_with_center

def draw_points_on_image(img_path, points, save_path="output_image.jpg", point_color=(0, 0, 255), point_size=5):
    """
    Draw specified points on the image.

    Parameters:
    - img_path: Path to the input image.
    - points: List or numpy array of points. Each point is represented as [x, y].
    - save_path: Path where the output image with points drawn will be saved.
    - point_color: Color of the points to be drawn. Default is red.
    - point_size: Size of the points.

    Returns:
    - None
    """
    
    # Read the image
    img = cv2.imread(img_path)
    
    # Validate if image is read successfully
    if img is None:
        raise ValueError("Image not found or unable to open.")
    # Draw each point on the image
    for car in points:
        for point in car:
            cv2.circle(img, (int(point[0]), int(point[1])), point_size, point_color, -1)
    
    # Save the image with points drawn on it
    cv2.imwrite(save_path, img)


In [6]:
cars = info_3d.bboxes_3d
print(cars)
corners = get_corners(cars, lidar2cam)
points = points_cam2img(corners, cam2img)
print(corners)
draw_points_on_image(img_path=img_path, points=points, save_path="output_image.jpg", point_color=(0, 0, 255), point_size=5)

CameraInstance3DBoxes(
    tensor([[ 2.0580e+01, -7.9915e+00,  8.2134e+01,  3.6595e-01,  1.7552e+00,
          4.6129e-01, -1.5958e+00],
        [-5.1892e+00, -2.7946e+01,  1.9222e+02,  4.1955e+00,  1.4736e+00,
          1.8690e+00, -1.5853e+00],
        [-1.7346e+00,  3.2832e+00,  2.3790e+01,  4.2404e+00,  1.4830e+00,
          2.0230e+00, -1.6060e+00],
        [-9.7301e+00, -1.6569e+01,  1.2978e+02,  4.2552e+00,  1.5313e+00,
          1.8960e+00,  1.5055e+00],
        [ 7.8163e-01, -3.0881e+00,  5.7809e+01,  6.9738e-01,  1.7748e+00,
          6.9320e-01, -4.0851e-02],
        [-3.2947e+00, -2.5933e+01,  1.8089e+02,  4.2140e+00,  1.5732e+00,
          1.8161e+00, -1.5853e+00],
        [-2.3218e+00, -1.7283e+01,  1.3339e+02,  4.2213e+00,  1.5024e+00,
          1.8683e+00, -1.5853e+00],
        [-1.0413e+01, -4.8058e+00,  6.6885e+01,  4.3101e+00,  1.7062e+00,
          1.9642e+00,  1.5055e+00],
        [-9.7736e+00,  1.2713e+00,  3.4580e+01,  4.5409e+00,  1.5320e+00,
          1.8975e+0